### Music Store




In [ ]:
import sqlite3
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/Bootcamp_Data_Science/DBs/music-store.db'
conn = sqlite3.connect(path)

In [ ]:
def execute_query(query):
  return pd.read_sql(query, conn)

In [ ]:
execute_query("SELECT * FROM sqlite_master WHERE type = 'table'")

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,artists,artists,2,CREATE TABLE artists\n(\n id INTEGER PRIMAR...
2,table,albums,albums,4,CREATE TABLE albums\n(\n id INTEGER PRIMARY...
3,table,employees,employees,5,CREATE TABLE employees\n(\n id INTEGER PRIM...
4,table,customers,customers,6,CREATE TABLE customers\n(\n id INTEGER PRIM...
5,table,genres,genres,7,CREATE TABLE genres\n(\n id INTEGER PRIMARY...
6,table,invoices,invoices,8,CREATE TABLE invoices\n(\n id INTEGER PRIMA...
7,table,media_types,media_types,9,CREATE TABLE media_types\n(\n id INTEGER PR...
8,table,tracks,tracks,10,CREATE TABLE tracks\n(\n id INTEGER PRIMARY...
9,table,invoice_lines,invoice_lines,11,CREATE TABLE invoice_lines\n(\n id INTEGER ...


Qual artista possui mais albuns?

In [ ]:
execute_query("""
SELECT ar.name AS artist, COUNT(*) AS number_of_albums FROM artists ar
INNER JOIN albums al ON ar.id = al.artist_id
GROUP BY ar.name
ORDER BY number_of_albums DESC
LIMIT 1
""")

,artist,number_of_albums
0,Iron Maiden,21


Existem clientes de quantos países?

In [ ]:
execute_query("SELECT DISTINCT COUNT(country) AS number_of_countries FROM customers")

,number_of_countries
0,59


Quantos clientes são brasileiros?

In [ ]:
execute_query("""
SELECT country, COUNT(country) AS customers FROM customers
WHERE country = 'Brazil'
""")

,country,customers
0,Brazil,5


Qual genero possui mais músicas?

In [ ]:
execute_query("""
SELECT g.name AS genre, COUNT(*) as number_of_music FROM genres as g
INNER JOIN tracks AS t ON g.id = t.genre_id
GROUP BY t.genre_id
ORDER BY number_of_music DESC
LIMIT 1
""")

,genre,number_of_music
0,Rock,1297


Qual genero possui mais albuns?

In [ ]:
execute_query("""
SELECT g.name AS genre, COUNT(*) AS number_of_albums FROM genres AS g
INNER JOIN tracks AS t ON g.id = t.genre_id
INNER JOIN albums AS al ON t.album_id = al.id
GROUP BY t.genre_id, t.album_id
ORDER BY number_of_albums DESC
LIMIT 1
""")

,genre,number_of_albums
0,Latin,34


Exiba os 5 clientes que mais fizeram pedidos

In [ ]:
execute_query("""
SELECT c.first_name, c.last_name, COUNT(i.customer_id) AS frequency FROM customers AS c
INNER JOIN invoices AS i ON c.id = i.customer_id
GROUP BY i.customer_id
ORDER BY frequency DESC
LIMIT 5
""")

,first_name,last_name,frequency
0,Luís,Gonçalves,7
1,Leonie,Köhler,7
2,François,Tremblay,7
3,Bjørn,Hansen,7
4,František,Wichterlová,7


Qual é o valor da maior compra?

In [ ]:
execute_query("""
SELECT * FROM invoices
ORDER BY total DESC
LIMIT 1""")

,id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
0,404,6,2011-11-13 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,25.86


Quais os emails dos 10 clientes que mais gastaram na loja?

In [ ]:
execute_query("""
SELECT c.email, i.customer_id, SUM(i.total) as total_cost FROM invoices as i
INNER JOIN customers as c ON i.customer_id = c.id
GROUP BY customer_id
ORDER BY total_cost DESC
LIMIT 10
""")

,email,customer_id,total_cost
0,hholy@gmail.com,6,49.62
1,ricunningham@hotmail.com,26,47.62
2,luisrojas@yahoo.cl,57,46.62
3,ladislav_kovacs@apple.hu,45,45.62
4,hughoreilly@apple.ie,46,45.62
5,jubarnett@gmail.com,28,43.62
6,fralston@gmail.com,24,43.62
7,fzimmermann@yahoo.de,37,43.62
8,astrid.gruber@apple.at,7,42.62
9,vstevens@yahoo.com,25,42.62


Qual o valor médio de cada compra?

In [ ]:
execute_query("SELECT AVG(total) AS invoice_average FROM invoices")

,invoice_average
0,5.651942


Quantas compras estão acima da média?

In [ ]:
execute_query("""SELECT COUNT(*) AS invoices_above_average FROM invoices
WHERE total > (SELECT AVG(total) FROM invoices)
""")

,invoices_above_average
0,179


Vamos fazer uma playlist com as 12 músicas mais vendidas! Quais foram elas? Quais os generos e os artistas?

In [ ]:
execute_query("""
SELECT a.name as artist, t.name as song, il.track_id, g.name as genre, COUNT(*) as purchases FROM invoice_lines il
INNER JOIN tracks t ON t.id = il.track_id
INNER JOIN genres g ON g.id = t.genre_id
INNER JOIN albums al ON al.id = t.album_id
INNER JOIN artists a ON a.id = al.artist_id
GROUP BY il.track_id
ORDER BY purchases DESC
LIMIT 12
""")

,artist,song,track_id,genre,purchases
0,Accept,Balls to the Wall,2,Rock,2
1,AC/DC,Inject The Venom,8,Rock,2
2,AC/DC,Snowballed,9,Rock,2
3,AC/DC,Overdose,20,Rock,2
4,Aerosmith,Deuces Are Wild,32,Rock,2
5,Alanis Morissette,Not The Doctor,48,Rock,2
6,Antônio Carlos Jobim,Por Causa De Você,66,Jazz,2
7,Apocalyptica,Welcome Home (Sanitarium),84,Metal,2
8,Black Sabbath,Snowblind,161,Metal,2
9,Black Sabbath,Cornucopia,162,Metal,2


Quais os gêneros mais vendido?

In [ ]:
execute_query("""
SELECT g.name as genre, COUNT(*) as songs_purchased FROM invoice_lines il
INNER JOIN tracks t ON t.id = il.track_id
INNER JOIN genres g ON g.id = t.genre_id
GROUP BY g.id
ORDER BY songs_purchased DESC
LIMIT 12
""")

,genre,songs_purchased
0,Rock,835
1,Latin,386
2,Metal,264
3,Alternative & Punk,244
4,Jazz,80
5,Blues,61
6,TV Shows,47
7,R&B/Soul,41
8,Classical,41
9,Reggae,30


Quais os artistas mais vendidos?

In [ ]:
execute_query("""
SELECT a.name as artist, COUNT(*) as songs_purchased FROM invoice_lines il
INNER JOIN tracks t ON t.id = il.track_id
INNER JOIN albums al ON al.id = t.album_id
INNER JOIN artists a ON a.id = al.artist_id
GROUP BY a.id
ORDER BY songs_purchased DESC
LIMIT 12
""")

,artist,songs_purchased
0,Iron Maiden,140
1,U2,107
2,Metallica,91
3,Led Zeppelin,87
4,Os Paralamas Do Sucesso,45
5,Deep Purple,44
6,Faith No More,42
7,Lost,41
8,Eric Clapton,40
9,R.E.M.,39


## Desafio do professor


* Precisamos definir 2 países para ações de marketing. Para um, iremos direcionar campanhas dos albums mais caros, nosso foco são os clientes que não se preocupam com o valor do pedido. 

* Para o outro grupo, que gasta menos, precisamos ser mais assertivos, recomendando albums mais baratos mas com músicas populares. 

* Qual seria sua estratégia? Quais países poderíamos atender? Quais albuns iremos recomendar para cada um dos países? Apresente a query (ou as queries), a tabela (ou as tabelas) e justifique suas escolhas. 

Álbuns mais caros





In [ ]:
execute_query("""
SELECT al.title, a.name, COUNT(*) as number_of_tracks, SUM(t.unit_price) as price FROM tracks t
INNER JOIN albums al ON al.id = t.album_id
INNER JOIN artists a ON a.id = al.artist_id
GROUP BY al.id
HAVING al.title NOT LIKE "%season%"
ORDER BY price DESC
LIMIT 10
""")

,title,name,number_of_tracks,price
0,Greatest Hits,Lenny Kravitz,57,56.43
1,Minha Historia,Chico Buarque,34,33.66
2,Unplugged,Eric Clapton,30,29.70
3,My Way: The Best Of Frank Sinatra [Disc 1],Frank Sinatra,24,23.76
4,Afrociberdelia,Chico Science & Nação Zumbi,23,22.77
5,Instant Karma: The Amnesty International Campa...,U2,23,22.77
6,Up An' Atom,Gene Krupa,22,21.78
7,Acústico,Titãs,22,21.78
8,International Superhits,Green Day,21,20.79
9,Acústico MTV,Os Paralamas Do Sucesso,21,20.79


Preço médio dos álbuns

In [ ]:
execute_query("""
SELECT AVG(albums_price.price) AS album_average_price
FROM (SELECT al.title, SUM(t.unit_price) as price 
FROM tracks t
INNER JOIN albums al ON al.id = t.album_id
GROUP BY al.id
HAVING al.title NOT LIKE "%season%") AS albums_price
""")

,album_average_price
0,9.702649


Músicas mais vendidas por album

In [ ]:
execute_query("""
SELECT al.title, a.name as artist, COUNT(*) as number_of_songs_purchased FROM invoice_lines il
INNER JOIN tracks t ON t.id = il.track_id
INNER JOIN albums al ON al.id = t.album_id
INNER JOIN artists a ON a.id = al.artist_id
GROUP BY al.id
ORDER BY number_of_songs_purchased DESC
LIMIT 10
""")

,title,artist,number_of_songs_purchased
0,Minha Historia,Chico Buarque,27
1,Greatest Hits,Lenny Kravitz,26
2,Unplugged,Eric Clapton,25
3,Acústico,Titãs,22
4,Greatest Kiss,Kiss,20
5,Prenda Minha,Caetano Veloso,19
6,"Chronicle, Vol. 2",Creedence Clearwater Revival,19
7,My Generation - The Very Best Of The Who,The Who,19
8,International Superhits,Green Day,18
9,"Chronicle, Vol. 1",Creedence Clearwater Revival,18


Países que tem a média mais cara por pedido

In [ ]:
execute_query("""
SELECT billing_country, AVG(total) as mean, COUNT(*) as frequency FROM invoices
GROUP BY billing_country
ORDER BY mean DESC, frequency DESC
LIMIT 10
""")

,billing_country,mean,frequency
0,Chile,6.660000,7
1,Hungary,6.517143,7
2,Ireland,6.517143,7
3,Czech Republic,6.445714,14
4,Austria,6.088571,7
5,Finland,5.945714,7
6,Netherlands,5.802857,7
7,India,5.789231,13
8,USA,5.747912,91
9,Norway,5.660000,7


Países que tem a média mais barata por pedido (com frequência de compra)



In [ ]:
execute_query("""
SELECT billing_country, AVG(total) as mean, COUNT(*) as frequency FROM invoices
GROUP BY billing_country
ORDER BY mean, frequency
LIMIT 10
""")

,billing_country,mean,frequency
0,Belgium,5.374286,7
1,Spain,5.374286,7
2,United Kingdom,5.374286,21
3,Argentina,5.374286,7
4,Australia,5.374286,7
5,Denmark,5.374286,7
6,Italy,5.374286,7
7,Poland,5.374286,7
8,Canada,5.427857,56
9,Brazil,5.431429,35


Qual o gênero mais comprado por país?




In [ ]:
execute_query("""
SELECT i.billing_country, g.name, COUNT(*) as count FROM invoices i
INNER JOIN invoice_lines il ON i.id = il.invoice_id
INNER JOIN tracks t ON t.id = il.track_id
INNER JOIN genres g ON g.id = t.genre_id
GROUP BY i.billing_country, g.name
ORDER BY i.billing_country, count DESC
""")

,billing_country,name,count
0,Argentina,Alternative & Punk,9
1,Argentina,Rock,9
2,Argentina,Latin,8
3,Argentina,Metal,7
4,Argentina,Easy Listening,2
...,...,...,...
232,United Kingdom,Jazz,4
233,United Kingdom,Hip Hop/Rap,3
234,United Kingdom,Pop,2
235,United Kingdom,R&B/Soul,2


Qual o artista mais comprado por país?

In [ ]:
execute_query("""
SELECT i.billing_country, a.name AS artist, COUNT(*) as purchases FROM invoices i
INNER JOIN invoice_lines il ON i.id = il.invoice_id
INNER JOIN tracks t ON t.id = il.track_id
INNER JOIN albums al ON al.id = t.album_id
INNER JOIN artists a ON a.id = al.artist_id
GROUP BY i.billing_country, a.name
ORDER BY i.billing_country, purchases DESC
""")

,billing_country,artist,purchases
0,Argentina,Metallica,6
1,Argentina,Foo Fighters,5
2,Argentina,Titãs,5
3,Argentina,Tim Maia,4
4,Argentina,Faith No More,2
...,...,...,...
699,United Kingdom,João Suplicy,1
700,United Kingdom,Lenny Kravitz,1
701,United Kingdom,Men At Work,1
702,United Kingdom,R.E.M. Feat. Kate Pearson,1


Gênero preferido do Chile

In [ ]:
execute_query("""
SELECT i.billing_country, g.name, COUNT(*) as purchases FROM invoices i
INNER JOIN invoice_lines il ON i.id = il.invoice_id
INNER JOIN tracks t ON t.id = il.track_id
INNER JOIN genres g ON g.id = t.genre_id
GROUP BY i.billing_country, g.name
HAVING i.billing_country = "Chile"
ORDER BY purchases DESC
""")

,billing_country,name,purchases
0,Chile,Rock,9
1,Chile,Latin,8
2,Chile,Drama,4
3,Chile,Easy Listening,3
4,Chile,Metal,3
5,Chile,Alternative & Punk,2
6,Chile,Blues,2
7,Chile,Science Fiction,2
8,Chile,TV Shows,2
9,Chile,Classical,1


Gênero preferido da Belgica

In [ ]:
execute_query("""
SELECT i.billing_country, g.name, COUNT(*) as purchases FROM invoices i
INNER JOIN invoice_lines il ON i.id = il.invoice_id
INNER JOIN tracks t ON t.id = il.track_id
INNER JOIN genres g ON g.id = t.genre_id
GROUP BY i.billing_country, g.name
HAVING i.billing_country = "Belgium"
ORDER BY purchases DESC
""")

,billing_country,name,purchases
0,Belgium,Rock,21
1,Belgium,Alternative & Punk,14
2,Belgium,R&B/Soul,2
3,Belgium,Metal,1


Artista preferido do Chile

In [ ]:
execute_query("""
SELECT i.billing_country, a.name AS artist, COUNT(*) as purchases FROM invoices i
INNER JOIN invoice_lines il ON i.id = il.invoice_id
INNER JOIN tracks t ON t.id = il.track_id
INNER JOIN albums al ON al.id = t.album_id
INNER JOIN artists a ON a.id = al.artist_id
GROUP BY i.billing_country, a.name
HAVING i.billing_country = "Chile"
ORDER BY purchases DESC
""")

,billing_country,artist,purchases
0,Chile,Led Zeppelin,6
1,Chile,Eric Clapton,4
2,Chile,Battlestar Galactica,3
3,Chile,Frank Sinatra,3
4,Chile,Gilberto Gil,3
5,Chile,Heroes,3
6,Chile,Lost,3
7,Chile,Bruce Dickinson,2
8,Chile,Creedence Clearwater Revival,2
9,Chile,Elis Regina,2


Artista preferido da Belgica

In [ ]:
execute_query("""
SELECT i.billing_country, a.name AS artist, COUNT(*) as purchases FROM invoices i
INNER JOIN invoice_lines il ON i.id = il.invoice_id
INNER JOIN tracks t ON t.id = il.track_id
INNER JOIN albums al ON al.id = t.album_id
INNER JOIN artists a ON a.id = al.artist_id
GROUP BY i.billing_country, a.name
HAVING i.billing_country = "Belgium"
ORDER BY purchases DESC
""")

,billing_country,artist,purchases
0,Belgium,Faith No More,9
1,Belgium,The Rolling Stones,5
2,Belgium,Aerosmith,4
3,Belgium,Smashing Pumpkins,4
4,Belgium,The Tea Party,3
5,Belgium,AC/DC,2
6,Belgium,Marvin Gaye,2
7,Belgium,R.E.M.,2
8,Belgium,The Cult,2
9,Belgium,The Who,2


Albuns dos 3 artistas preferidos ordenados por preço mais baixo

In [ ]:
execute_query(""" 
SELECT a.name AS artist, al.title, SUM(unit_price) as price FROM tracks t
INNER JOIN albums al ON al.id = t.album_id
INNER JOIN artists a ON a.id = al.artist_id
GROUP BY al.id
HAVING a.name = "Faith No More" OR
  a.name = "The Rolling Stones" OR
  a.name = "Aerosmith"
ORDER BY 3
""")

,artist,title,price
0,Faith No More,The Real Thing,10.89
1,Faith No More,Album Of The Year,11.88
2,The Rolling Stones,"Hot Rocks, 1964-1971 (Disc 1)",11.88
3,Faith No More,Angel Dust,13.86
4,The Rolling Stones,No Security,13.86
5,Aerosmith,Big Ones,14.85
6,Faith No More,King For A Day Fool For A Lifetime,14.85
7,The Rolling Stones,Voodoo Lounge,14.85


Músicas mais vendidas da Belgica

In [ ]:
execute_query("""
SELECT i.billing_country, a.name as artist, t.name as song, al.title as album, COUNT(*) as purchases FROM invoice_lines il
INNER JOIN invoices i ON i.id = il.invoice_id
INNER JOIN tracks t ON t.id = il.track_id
INNER JOIN albums al ON al.id = t.album_id
INNER JOIN artists a ON a.id = al.artist_id
GROUP BY il.track_id
HAVING i.billing_country = "Belgium"
ORDER BY purchases DESC
LIMIT 10
""")

,billing_country,artist,song,album,purchases
0,Belgium,Faith No More,She Loves Me Not,Album Of The Year,2
1,Belgium,Faith No More,Caffeine,Angel Dust,2
2,Belgium,Faith No More,Easy,Angel Dust,2
3,Belgium,Faith No More,What A Day,King For A Day Fool For A Lifetime,2
4,Belgium,Marvin Gaye,Save The Children,Seek And Shall Find: More Of The Best (1963-1981),2
5,Belgium,Smashing Pumpkins,Cherub Rock,Rotten Apples: Greatest Hits,2
6,Belgium,Smashing Pumpkins,Bullet With Butterfly Wings,Rotten Apples: Greatest Hits,2
7,Belgium,AC/DC,Dog Eat Dog,Let There Be Rock,1
8,Belgium,Aerosmith,Love In An Elevator,Big Ones,1
9,Belgium,Aerosmith,Janie's Got A Gun,Big Ones,1


Para a campanha voltada para o país que nâo se importa com preços, o país escolhido seria o Chile, pois é o país com a maior média de gasto por compra. O album escolhido para a campanha seria o Unplugged	do Eric Clapton. Este é o terceiro album mais caro, mas Eric Clapton é o segundo artista mais vendido para o Chile.	

Para a campanha voltada para o país que gasta menos, foi escolhida a Bélgica, por ter a menor média por compra e a menor frequência. O álbum escolhido para essa promoção foi o Album Of The Year do Faith No More. Faith No More é a banda mais vendida por clientes da Bélgica e a música mais vendida está neste álbum. Album Of The Year é o segundo álbum mais barato dentre os três artistas preferidos dos belgas e custa um pouco mais de 2 dólares acima do preço médio geral de álbuns.